In [1]:
# 导入相关库
import os
import pandas as pd
from tqdm import *
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import accuracy_score
import time
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.model_selection import StratifiedKFold
from gensim.models import FastText, Word2Vec
import re
from keras.layers import *
from keras.models import *
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from keras.callbacks import *
from keras.layers.advanced_activations import LeakyReLU, PReLU
import keras.backend as K
from keras.optimizers import *
from keras.utils import to_categorical
import tensorflow as tf
import random as rn
import gc
import logging
import gensim
import jieba
tqdm.pandas()
os.environ['PYTHONHASHSEED'] = '0'
# 显卡使用（如没显卡需要注释掉）
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
np.random.seed(1024)
rn.seed(1024)
tf.set_random_seed(1024)
path="data/"
os.listdir("data/")

Using TensorFlow backend.


['sample.csv',
 'apptype_id_name.txt',
 'app_desc.dat',
 'submit.csv',
 '.ipynb_checkpoints',
 'apptype_train.dat']

In [2]:
data=[]
with open(path+ "app_desc.dat", encoding='utf-8')as f:
    for line in f.readlines():
        data.append(line.strip("\r\n").split("\t"))
app_desc = pd.DataFrame(data,columns=['id', 'content'])

apptype_id_name = pd.read_table(path + 'apptype_id_name.txt', header=None)
apptype_id_name.columns = ['appid', 'appshow']
sample = pd.read_csv(path + 'sample.csv')
f = open(path + 'apptype_train.dat', encoding='utf-8')
apptype_train = pd.DataFrame()
a1 = []
a2 = []
a3 = []
for i in f.read().split('\n'):
    if len(i.split('\t')[1].split('|')) == 1: 
        a1.append(i.split('\t')[0])
        a2.append(i.split('\t')[1])
        a3.append(i.split('\t')[2])
    else:
        a1.append(i.split('\t')[0])
        a3.append(i.split('\t')[2])
        a2.append(i.split('\t')[1].split('|')[0])
        a1.append(i.split('\t')[0])
        a3.append(i.split('\t')[2])
        a2.append(i.split('\t')[1].split('|')[1])
apptype_train['id'] = a1
apptype_train['content'] = a3
apptype_train['label'] = a2
f.close()

In [3]:
# 需要用到的函数
from keras import backend as K


class Lookahead(object):
    """Add the [Lookahead Optimizer](https://arxiv.org/abs/1907.08610) functionality for [keras](https://keras.io/).
    """

    def __init__(self, k=5, alpha=0.5):
        self.k = k
        self.alpha = alpha
        self.count = 0

    def inject(self, model):
        """Inject the Lookahead algorithm for the given model.
        The following code is modified from keras's _make_train_function method.
        See: https://github.com/keras-team/keras/blob/master/keras/engine/training.py#L497
        """
        if not hasattr(model, 'train_function'):
            raise RuntimeError('You must compile your model before using it.')

        model._check_trainable_weights_consistency()

        if model.train_function is None:
            inputs = (model._feed_inputs +
                      model._feed_targets +
                      model._feed_sample_weights)
            if model._uses_dynamic_learning_phase():
                inputs += [K.learning_phase()]
            fast_params = model._collected_trainable_weights

            with K.name_scope('training'):
                with K.name_scope(model.optimizer.__class__.__name__):
                    training_updates = model.optimizer.get_updates(
                        params=fast_params,
                        loss=model.total_loss)
                    slow_params = [K.variable(p) for p in fast_params]
                fast_updates = (model.updates +
                                training_updates +
                                model.metrics_updates)

                slow_updates, copy_updates = [], []
                for p, q in zip(fast_params, slow_params):
                    slow_updates.append(K.update(q, q + self.alpha * (p - q)))
                    copy_updates.append(K.update(p, q))

                # Gets loss and metrics. Updates weights at each call.
                fast_train_function = K.function(
                    inputs,
                    [model.total_loss] + model.metrics_tensors,
                    updates=fast_updates,
                    name='fast_train_function',
                    **model._function_kwargs)

                def F(inputs):
                    self.count += 1
                    R = fast_train_function(inputs)
                    if self.count % self.k == 0:
                        K.batch_get_value(slow_updates)
                        K.batch_get_value(copy_updates)
                    return R
                
                model.train_function = F

class AdamW(Optimizer):
    def __init__(self, lr=0.001, beta_1=0.9, beta_2=0.999, weight_decay=1e-4,  # decoupled weight decay (1/4)
                 epsilon=1e-8, decay=0., **kwargs):
        super(AdamW, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.lr = K.variable(lr, name='lr')
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_2, name='beta_2')
            self.decay = K.variable(decay, name='decay')
            # decoupled weight decay (2/4)
            self.wd = K.variable(weight_decay, name='weight_decay')
        self.epsilon = epsilon
        self.initial_decay = decay

    @interfaces.legacy_get_updates_support
    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]
        wd = self.wd  # decoupled weight decay (3/4)

        lr = self.lr
        if self.initial_decay > 0:
            lr *= (1. / (1. + self.decay * K.cast(self.iterations,
                                                  K.dtype(self.decay))))

        t = K.cast(self.iterations, K.floatx()) + 1
        lr_t = lr * (K.sqrt(1. - K.pow(self.beta_2, t)) /
                     (1. - K.pow(self.beta_1, t)))

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        self.weights = [self.iterations] + ms + vs

        for p, g, m, v in zip(params, grads, ms, vs):
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)
            # decoupled weight decay (4/4)
            p_t = p - lr_t * m_t / (K.sqrt(v_t) + self.epsilon) - lr * wd * p

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))
            new_p = p_t

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates

    def get_config(self):
        config = {'lr': float(K.get_value(self.lr)),
                  'beta_1': float(K.get_value(self.beta_1)),
                  'beta_2': float(K.get_value(self.beta_2)),
                  'decay': float(K.get_value(self.decay)),
                  'weight_decay': float(K.get_value(self.wd)),
                  'epsilon': self.epsilon}
        base_config = super(AdamW, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


from keras.engine.topology import Layer
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim




In [ ]:
### 读入数据(分词)
data = pd.concat([apptype_train, app_desc], axis=0, sort=False)
data['word_cut'] = data['content'].progress_apply(lambda row:' '.join(jieba.lcut(str(row))))
data['char_cut'] = data['content'].progress_apply(lambda row:' '.join(list(row)))
data.head(5)

  0%|          | 0/131133 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.761 seconds.
Prefix dict has been built succesfully.
 70%|██████▉   | 91679/131133 [01:50<00:46, 849.91it/s]

In [ ]:
data['word_len'] = data['word_cut'].apply(lambda row:len(str(row).split(' ')))
data['char_len'] = data['char_cut'].apply(lambda row:len(str(row).split(' ')))
print('word长度:',np.percentile(data['word_len'], 95))
print('char长度:',np.percentile(data['char_len'], 95))

In [ ]:
### Tokenizer 序列化文本
def set_tokenizer(docs, split_char=' ', max_len=100):
    '''
    输入
    docs:文本列表
    split_char:按什么字符切割
    max_len:截取的最大长度
    
    输出
    X:序列化后的数据
    word_index:文本和数字对应的索引
    '''
    tokenizer = Tokenizer(lower=False, char_level=False, split=split_char)
    tokenizer.fit_on_texts(docs)
    X = tokenizer.texts_to_sequences(docs)
    maxlen = max_len
    X = pad_sequences(X, maxlen=maxlen, value=0)
    word_index=tokenizer.word_index
    return X, word_index

### 做embedding 这里采用word2vec 可以换成其他例如（glove词向量）
def trian_save_word2vec(docs, embed_size=300, save_name='w2v.txt', split_char=' '):
    '''
    输入
    docs:输入的文本列表
    embed_size:embed长度
    save_name:保存的word2vec位置
    
    输出
    w2v:返回的模型
    '''
    input_docs = []
    for i in docs:
        input_docs.append(i.split(split_char))
    logging.basicConfig(
    format='%(asctime)s:%(levelname)s:%(message)s', level=logging.INFO)
    w2v = Word2Vec(input_docs, size=embed_size, sg=1, window=8, seed=1017, workers=24, min_count=1, iter=10)
    w2v.save(save_name)
    print("w2v model done")
    return w2v

# 得到embedding矩阵
def get_embedding_matrix(word_index, embed_size=300, Emed_path="w2v_300.txt"):
    embeddings_index = Word2Vec.load(Emed_path)
    nb_words = len(word_index)+1
    embedding_matrix = np.zeros((nb_words, embed_size))
    count = 0
    for word, i in tqdm(word_index.items()):
        if i >= nb_words:
            continue
        try:
            embedding_vector = embeddings_index[word]
        except:
            embedding_vector = np.zeros(embed_size)
            count += 1
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector  
    print("null cnt",count)
    return embedding_matrix

# 得到fasttext矩阵
def load_fasttext(word_index, path):  
    count=0
    null_list=[]
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(path, encoding='utf-8') if len(o)>100)

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index
    nb_words =  len(word_index)+1
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= nb_words: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
        else:
            null_list.append(word)
            count+=1
    print("null cnt:",count)
    return embedding_matrix

def get_embedding_matrix_txt(word_index,embed_size=200,Emed_path="w2v_300.txt"):
    embeddings_index = gensim.models.KeyedVectors.load_word2vec_format(
        Emed_path, binary=False)
    nb_words = len(word_index)+1
    embedding_matrix = np.zeros((nb_words, embed_size))
    count = 0
    for word, i in tqdm(word_index.items()):
        if i >= nb_words:
            continue
        try:
            embedding_vector = embeddings_index[word]
        except:
            embedding_vector = np.zeros(embed_size)
            count += 1
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    print("null cnt",count)
    return embedding_matrix

In [ ]:
# word_cut 330
# char_cut 580
text_1_list = list(data['word_cut'])
text_3_list = list(data['char_cut'])

print('开始序列化')
x1, index_1 = set_tokenizer(text_1_list, split_char=' ', max_len=330)
x3, index_3 = set_tokenizer(text_3_list, split_char=' ', max_len=580)
print('序列化完成')
gc.collect()

# trian_save_word2vec(text_1_list, save_name='w2v_model/word_w2v_300.txt', split_char=' ')
# gc.collect()
# trian_save_word2vec(text_3_list, save_name='w2v_model/char_w2v_300.txt', split_char=' ')
# gc.collect()
# emb1_extend_1 = load_fasttext(index_1, path='w2v_model/cc.zh.300.vec')
# emb1_extend_2 = get_embedding_matrix_txt(index_1, embed_size=200, Emed_path="w2v_model/Tencent_AILab_ChineseEmbedding.txt")
# np.save('tmp/emb1_extend_1', emb1_extend_1)
# np.save('tmp/emb1_extend_2', emb1_extend_2)

# 得到emb矩阵
emb1 = get_embedding_matrix(index_1, Emed_path='w2v_model/word_w2v_300.txt')
emb1_extend_1 = np.load('tmp/emb1_extend_1.npy')
emb1_extend_2 = np.load('tmp/emb1_extend_2.npy')
emb1 = np.concatenate((emb1, emb1_extend_1, emb1_extend_2), axis=1)

emb3 = get_embedding_matrix(index_3, Emed_path='w2v_model/char_w2v_300.txt')
gc.collect()

In [ ]:
# 将feature作为输入进行处理(这个feature一定要替换掉nan与inf)
# 这个feature的顺序一定要对应之前的feature的顺序，保证每条是对应的
feature = data[['word_len', 'char_len']]
feature = feature.fillna(-1)
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
ss.fit(feature)
hin_feature = ss.transform(feature)
num_feature_input = hin_feature.shape[1]

In [ ]:
# 区分开train和valid,test
# 这里是假设三输入
data['label'] = data['label'].fillna(-1)
train_data = data[data['label']!=-1]
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
labelencoder.fit(train_data['label'])
train_data['label'] = labelencoder.transform(train_data['label'])
train_input_1 = x1[:len(train_data)]
test_input_1 = x1[len(train_data):]
train_input_3 = x3[:len(train_data)]
test_input_3 = x3[len(train_data):]
train_input_5 = hin_feature[:len(train_data)]
test_input_5 = hin_feature[len(train_data):]
label = to_categorical(train_data['label'])

In [ ]:
from keras.initializers import *

def model_conv(emb1, emb3, num_feature_input):
    '''
    注意这个inputs
    seq1、seq2分别是两个输入
    hin是feature层输入
    是否做emb可选可不选，
    这个就是我们之前训练已经得到的用于embedding的（embedding_matrix1， embedding_matrix2）
    '''
    K.clear_session()

    emb_layer_1 = Embedding(
        input_dim=emb1.shape[0],
        output_dim=emb1.shape[1],
        weights=[emb1],
        input_length=330,
        trainable=False
    )
    
    emb_layer_3 = Embedding(
        input_dim=emb3.shape[0],
        output_dim=emb3.shape[1],
        weights=[emb3],
        input_length=580,
        trainable=False
    )
    
    
    seq1 = Input(shape=(330,))
    seq3 = Input(shape=(580,))    
    
    x1 = emb_layer_1(seq1)
    x3 = emb_layer_3(seq3)
    
    sdrop=SpatialDropout1D(rate=0.2)

    x1 = sdrop(x1)
    x3 = sdrop(x3)
     
    x = Dropout(0.2)(Bidirectional(CuDNNGRU(256, return_sequences=True))(x1))
    semantic = TimeDistributed(Dense(100, activation="tanh"))(x)
    merged_1 = Lambda(lambda x: K.max(x, axis=1), output_shape=(100,))(semantic)
    
    x = Dropout(0.2)(Bidirectional(CuDNNGRU(256, return_sequences=True))(x3))
    semantic = TimeDistributed(Dense(100, activation="tanh"))(x)
    merged_3 = Lambda(lambda x: K.max(x, axis=1), output_shape=(100,))(semantic)
    
    hin = Input(shape=(num_feature_input, ))
    htime = Dense(16, activation='relu')(hin)
    
    x = concatenate([merged_1, merged_3, htime])
    
    x = Dropout(0.2)(Activation(activation="relu")(BatchNormalization()(Dense(1000)(x))))
    x = Activation(activation="relu")(BatchNormalization()(Dense(500)(x)))
    pred = Dense(125, activation='softmax')(x)
    model = Model(inputs=[seq1, seq3, hin], outputs=pred)
    model.compile(loss='categorical_crossentropy',
                  optimizer=AdamW(lr=0.001,weight_decay=0.08,),metrics=["accuracy"])
    lookahead = Lookahead(k=5, alpha=0.5) # Initialize Lookahead
    lookahead.inject(model) # add into model
    return model
gc.collect()

In [ ]:
skf = StratifiedKFold(n_splits=10, random_state=1017, shuffle=True)
sub = np.zeros((test_input_5.shape[0], 125))
oof_pred = np.zeros((train_input_5.shape[0], 125))
score = []
count = 0
version = 'v1'
if not os.path.exists("model/"+ version + "/"):
    os.mkdir("model/"+ version + "/")

for i, (train_index, test_index) in enumerate(skf.split(train_input_5, train_data['label'])):
    print("FOLD | ", count+1)
    print("###"*35)
    gc.collect()
    filepath = "model/"+ version + "/%d.h5" % count
    checkpoint = ModelCheckpoint(
        filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max',save_weights_only=True)
    reduce_lr = ReduceLROnPlateau(
        monitor='val_acc', factor=0.5, patience=3, min_lr=0.0001, verbose=1)
    earlystopping = EarlyStopping(
        monitor='val_acc', min_delta=0.0001, patience=5, verbose=1, mode='max')
    callbacks = [checkpoint, reduce_lr, earlystopping]
    model_age = model_conv(emb1, emb3, num_feature_input)
    if count==0:model_age.summary()
    x1_tr, x1_va = np.array(train_input_1)[train_index], np.array(train_input_1)[test_index]    
    x3_tr, x3_va = np.array(train_input_3)[train_index], np.array(train_input_3)[test_index]
    x5_tr, x5_va = np.array(train_input_5)[train_index], np.array(train_input_5)[test_index]
    y_tr, y_va = label[train_index], label[test_index]
    
    hist = model_age.fit([x1_tr, x3_tr, x5_tr],
                         y_tr, batch_size=512, epochs=50, 
                         validation_data=([x1_va, x3_va, x5_va], y_va),
                         callbacks=callbacks, verbose=1, shuffle=True)

    model_age.load_weights(filepath)
    oof_pred[test_index] = model_age.predict([x1_va, x3_va, x5_va],batch_size=1024,verbose=1)
    sub += model_age.predict([test_input_1, test_input_3, test_input_5],batch_size=1024,verbose=1)/skf.n_splits
    score.append(np.max(hist.history['val_acc']))
    count += 1
print('acc:', np.mean(score))

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


FOLD |  1
#########################################################################################################
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 330)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 580)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 330, 800)     225556000   input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 580, 300)     2626200     input_2[0][0]              

In [79]:
new_pred = np.argsort(-oof_pred, axis=1)
new_pred = new_pred[:,:2]
in_it = 0
for i, value in enumerate(list(train_data['label'])):
    if value in list(new_pred[i, :2]):
        in_it += 1
in_it/len(train_data)

0.7629279886940322

In [80]:
put_result = np.argsort(-sub, axis=1)
label_1 = put_result[:,0]
label_2 = put_result[:,1]
data['label'] = data['label'].fillna(-1)
test_data = data[data['label']==-1]
result = pd.DataFrame()
result['id'] = list(test_data['id'])
result['label1'] = list(labelencoder.inverse_transform(label_1))
result['label2'] = list(labelencoder.inverse_transform(label_2))
result.to_csv('result/v1.csv', index=False)

In [81]:
tr = pd.DataFrame(oof_pred)
te = pd.DataFrame(sub)
stack_result = pd.concat([tr, te], axis=0, sort=False)
new_columns = []
for i in stack_result.columns:
    new_columns.append(int(i))
new_columns = labelencoder.inverse_transform(new_columns)
stack_result.columns = new_columns
stack_result['id'] = data['id']
stack_result.to_csv('result/stack_v1.csv', index=False)